#### 开始之前，导入numpy、pandas包和数据

In [2]:
# 导入基本库
import numpy as np
import pandas as pd
text = pd.read_csv('result.csv')
text.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 2 第二章：数据重构


## 第一部分：数据聚合与运算

### 2.6 数据运用

#### 2.6.1 任务一：通过教材《Python for Data Analysis》P303、Google or anything来学习了解GroupBy机制

In [5]:
# df.groupby('key1')[['data2']]
# df.groupby('key1')['data1']
# df[data1'].groupby(df['key1'])
# df[[data2']].groupby(df['key1'])

#### 2.4.2：任务二：计算泰坦尼克号男性与女性的平均票价

In [54]:

df  = text['Fare'].groupby(text['Sex'])
means = df.mean()
means

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64

在了解GroupBy机制之后，运用这个机制完成一系列的操作，来达到我们的目的。

下面通过几个任务来熟悉GroupBy机制。

#### 2.4.3：任务三：统计泰坦尼克号中男女的存活人数

In [13]:

text.groupby('Sex').size()

Sex
female    314
male      577
dtype: int64

In [51]:
one = text.groupby(['Pclass','Sex']).count()[['PassengerId']].reset_index()
one = one.pivot_table(index='Pclass',columns='Sex')
one.columns = [item[1] for item in one.columns]
one['female_ratio'] = one.apply(lambda x: x.female/x.sum(),axis=1 )
one['male_ratio'] = one.apply(lambda x: x.male/x.sum(),axis=1 )


In [52]:
one

,female,male,female_ratio,male_ratio
Pclass,,,,
1,94,122,0.435185,0.563679
2,76,108,0.413043,0.585642
3,144,347,0.293279,0.706299


In [11]:

survived_sex = text['Survived'].groupby(text['Sex']).sum()
survived_sex.head()

Sex
female    233
male      109
Name: Survived, dtype: int64

In [48]:
survived_sex = text.groupby('Sex').sum()[['Survived']]
survived_sex.head()

/var/folders/nm/41yw47353z12bzs83s_xfcl80000gn/T/ipykernel_54758/3212473813.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  survived_sex = text.groupby('Sex').sum()[['Survived']]


,Survived
Sex,
female,233
male,109


#### 2.4.4：任务四：计算客舱不同等级的存活人数

In [3]:
survived_pclass = text['Survived'].groupby(text['Pclass'])
survived_pclass.sum()

Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64

### 思考
1. 在男女比例为`577：314`的情况下，最后存活下来的比例却是反比`109:233`.
2. 女性虽然跟男性一样分布于不同等级的`pclass`，但等级越高的女性比例越高,男性缺与之相反.
3. 在不同位置调用函数的参数会得到`series`(一维）或者`dataframe`（二维）结果.
4. `pclass`的等级越高，存活的比例越高.
5. 还可以通过更多的数据分析比较得到信息，比如获救人数中女性的存活跟`Pclass`是否有关联.

【思考】从任务二到任务三中，这些运算可以通过agg()函数来同时计算。并且可以使用rename函数修改列名。你可以按照提示写出这个过程吗？


#### 2.4.5：任务五：统计在不同等级的票中的不同年龄的船票花费的平均值

In [50]:
text.groupby('Sex').agg({'Fare': 'mean', 'Pclass': 'count'}).rename(columns=
                            {'Fare': 'mean_fare', 'Pclass': 'count_pclass'})


,mean_fare,count_pclass
Sex,,
female,44.479818,314
male,25.523893,577


#### 2.4.6：任务六：将任务二和任务三的数据合并，并保存到sex_fare_survived.csv

In [55]:
result = pd.merge(means,survived_sex,on='Sex')
result

,Fare,Survived
Sex,,
female,44.479818,233
male,25.523893,109


In [56]:
result.to_csv('sex_fare_survived.csv')

#### 2.4.7：任务七：得出不同年龄的总的存活人数，然后找出存活人数最多的年龄段，最后计算存活人数最高的存活率（存活人数/总人数）


In [59]:
text.groupby(['Pclass','Age'])['Fare'].mean().head()

Pclass  Age  
1       0.92     151.5500
        2.00     151.5500
        4.00      81.8583
        11.00    120.0000
        14.00    120.0000
Name: Fare, dtype: float64

In [65]:
# 写入代码
survived_age = text.groupby('Age')['Survived'].sum()
survived_age.head()

Age
0.42    1
0.67    1
0.75    2
0.83    2
0.92    1
Name: Survived, dtype: int64

In [69]:

survived_age[survived_age.values==survived_age.max()]


Age
24.0    15
Name: Survived, dtype: int64

In [67]:
#计算存活总人数，对数字进行字符串输出
_sum = text['Survived'].sum()
print("sum of person:"+str(_sum))
precent =survived_age.max()/_sum
print("最大存活率："+str(precent))

sum of person:342
最大存活率：0.043859649122807015
